In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ReLU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import os

# Load data
csv_path = os.path.join('..', 'Data', 'Diagested_data', 'Numeric_resume_data.csv')
data = pd.read_csv(csv_path)
df = pd.DataFrame(data)

# Define trait columns
trait_columns = [
    'Leadership', 'Communication', 'Teamwork', 'Problem Solving', 'Creativity',
    'Adaptability', 'Work Ethic', 'Time Management', 'Interpersonal Skills', 
    'Attention to Detail', 'Initiative', 'Analytical Thinking', 'Emotional Intelligence', 
    'Integrity', 'Resilience', 'Cultural Awareness', 'Programming Languages', 
    'Technical Skills', 'Office Tools'
]

# Convert job titles to numerical labels
job_titles = df['Job Title'].unique()
job_domains = df[['Job Title', 'Domain']].drop_duplicates().set_index('Job Title')['Domain'].to_dict()
job_title_to_index = {title: idx for idx, title in enumerate(job_titles)}
index_to_job_title = {idx: title for title, idx in job_title_to_index.items()}
df['Job Label'] = df['Job Title'].map(job_title_to_index)

# Prepare feature and target arrays
X = df[trait_columns].values
y = df['Job Label'].values

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Augment data with Gaussian noise
def add_gaussian_noise(X, noise_factor=0.05):
    noise = np.random.normal(loc=0.0, scale=noise_factor, size=X.shape)
    X_noisy = X + noise
    return np.clip(X_noisy, 0., 1.)

X_augmented = add_gaussian_noise(X_scaled)

# Combine original and augmented data
X_combined = np.vstack((X_scaled, X_augmented))
y_combined = np.hstack((y, y))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(job_titles)
lb = LabelBinarizer()
lb.fit(y_combined)  # Fit on the entire set of labels to ensure it has all classes
y_train_onehot = lb.transform(y_train)
y_test_onehot = lb.transform(y_test)

# Define a neural network model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],)),
    ReLU(),
    Dropout(0.5),
    Dense(64),
    ReLU(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with augmented data
model.fit(X_train, y_train_onehot, epochs=1000, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_onehot)
print(f'Test Accuracy: {accuracy:.2f}')

# Define the save path for the model
model_save_path = os.path.join('..', 'Code', 'Pickles', 'job_recommendation_algorithm.h5')

# Ensure the directory exists
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Save the model to the specified file
model.save(model_save_path)


2024-08-29 12:23:34.955325: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 12:23:35.563810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 12:23:35.564338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 12:23:35.631108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 12:23:35.819364: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/1000


2024-08-29 12:23:42.533503: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-29 12:23:43.732717: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc055d13930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-29 12:23:43.732822: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-08-29 12:23:43.817046: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-29 12:23:43.972357: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1724923424.270427    5253 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


33/33 [==============================] - 4s 25ms/step - loss: 7.3494 - accuracy: 0.0000e+00 - val_loss: 7.3456 - val_accuracy: 0.0038
Epoch 2/1000
33/33 [==============================] - 1s 17ms/step - loss: 7.2700 - accuracy: 0.0052 - val_loss: 7.3829 - val_accuracy: 0.0019
Epoch 3/1000
33/33 [==============================] - 1s 19ms/step - loss: 7.1481 - accuracy: 0.0052 - val_loss: 7.5369 - val_accuracy: 0.0019
Epoch 4/1000
33/33 [==============================] - 1s 17ms/step - loss: 6.9570 - accuracy: 0.0105 - val_loss: 7.7829 - val_accuracy: 0.0057
Epoch 5/1000
33/33 [==============================] - 1s 17ms/step - loss: 6.7604 - accuracy: 0.0067 - val_loss: 7.9334 - val_accuracy: 0.0057
Epoch 6/1000
33/33 [==============================] - 1s 17ms/step - loss: 6.5341 - accuracy: 0.0143 - val_loss: 8.0849 - val_accuracy: 0.0057
Epoch 7/1000
33/33 [==============================] - 1s 17ms/step - loss: 6.3043 - accuracy: 0.0157 - val_loss: 8.2774 - val_accuracy: 0.0057
Epoch 8/

/home/yair7799/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
